In [1]:
# IMPORT 

import csv
from biomart import BiomartServer
import pandas as pd
import openpyxl

In [2]:
# DATASET

## biomart_server
server = BiomartServer('http://www.ensembl.org/biomart')
dataset = server.datasets['mmusculus_gene_ensembl']

In [3]:
# FUNCTIONS

def gene_dictionary(index,
                    gene_name,
                    gene_list_number,
                    gene_list_id,
                    source,
                    ensembl_gene_id,
                    ensembl_transcript_id,
                    refseq_mrna,
                    hgnc_symbol,
                    organism,
                    alias,
                    direction_string):
    
    dictionary = {
        'Index' : index,
        'Gene_name' : gene_name,
        'Gene_list_number' : gene_list_number,
        'Gene_list_index' : gene_list_id,
        'Source' : source,
        'Ensembl_gene_id' : ensembl_gene_id,
        'Ensembl_transcript_id' : ensembl_transcript_id,
        'RefSeq_mRNA_id' : refseq_mrna,
        'Organism' : organism,
        'HGNC_symbol' : hgnc_symbol,
        'Alias' : alias,
        'Info' : direction_string
    }
    
    return dictionary


  
def biomartParameters(mgi_symbol):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'mgi_symbol':[mgi_symbol]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values   


def biomartHumanOrthologs(mgi_symbol):
    attributes = ['hsapiens_homolog_associated_gene_name']
    filters = {'mgi_symbol':[mgi_symbol]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values 


def alias_and_official(ls_notResponse,ls_row_10,ls_row_2):

    ls_response = []
    ls_response2 = []
    
    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                t_strings = temp.split("|")
                if word in t_strings:
                    ls_response.append([word, i, ls_row_1[i]])

    ls_response.insert(0, ['gene_name','alias_index','mgi_id'])         # print(len(ls_response))


    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                if word == temp:
                    ls_response2.append([word, i, ls_row_1[i]])
                    

    ls_response.insert(0, ['gene_name','official_index','mgi_id'])      # print(len(ls_response2))

    ls_response_3 = ls_response
    
    for response in ls_response2:
        if response not in ls_response_3:
            ls_response_3.append(response)
        else: print(response)

    ls_response_3.pop(0)
    ls_response_3.pop(0)
    ls_response.insert(0, ['gene_name','official/alias_index','mgi_id'])
    
    return ls_response_3

def biomartParameters_mgi(mgi_id):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']
    filters = {'mgi_id':[mgi_id]}       # gene_name = mgi_id
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values   


def biomartHumanOrthologs_mgi(mgi_id):
    attributes = ['hsapiens_homolog_associated_gene_name']
    
    filters = {'mgi_id':[mgi_id]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values 


def updateCellswithAlias(mgi_file_path, dictionary_file_path, file_path):
    
    # Load MGI file and workbook
    wb_mgi = openpyxl.load_workbook(mgi_file_path)
    ws_mgi = wb_mgi.active

    ls_row_alias = [ws_mgi.cell(row=i,column=10).value for i in range(2,ws_mgi.max_row+1)]
    ls_row_g_name = [ws_mgi.cell(row=i,column=2).value for i in range(2,ws_mgi.max_row+1)] 

    # Load dictionary file and workbook
    workbook = openpyxl.load_workbook(dictionary_file_path)
    sheet = workbook.active

    ls_row_2 = [sheet.cell(row=i,column=2).value for i in range(2,sheet.max_row+1)]
    ls_row_9 = [sheet.cell(row=i,column=11).value for i in range(2,sheet.max_row+1)]

    
    length = len(ls_row_2) + 2

    # Modify cells
    for i in range(length):
        K_cel = 'K' + str(i + 2)

        for k in range(len(ls_row_g_name)):
            try:
                if str(ls_row_g_name[k]).lower() == str(ls_row_2[i]).lower():
                    if ls_row_alias[k] is not None:
                        sheet[K_cel] = ls_row_alias[k]
                    else:
                        continue
            except:
                continue

    workbook.save(file_path)
    
    
def updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path):
    
    wb_info = openpyxl.load_workbook(source_file_path)
    ws_info = wb_info[source_sheet]
    ls_row_g_name = [ws_info.cell(row=i, column=3).value for i in range(2, ws_info.max_row + 1)]


    wb_alias = openpyxl.load_workbook(alias_file_path)
    ws_alias = wb_alias.active
    ls_row_2 = [ws_alias.cell(row=i, column=2).value for i in range(2, ws_alias.max_row + 1)]


    matched_indices = set()               # to keep track of matched id

    for j, (g_name, row_2) in enumerate(zip(ls_row_g_name, ls_row_2), start=2):
        if j not in matched_indices and str(g_name).lower() == str(row_2).lower():
            print(str(g_name).lower(), str(row_2).lower())
            L_cel = 'L' + str(j)
            tempInfo = str(ws_alias[L_cel].value)
            matched_indices.add(j)        # add matched index to set - avoid duplicate
            

### the number and type of items in the INFO column depends on the source publication

            ws_alias[L_cel] = ''
            ws_alias[L_cel] = ('Regulation:' + str(tempInfo).upper() + '|'
                            + 'Log2Ratio:' + str(ws_info['E' + str(j)].value) + '|'
                            + 'tissue:' + str(ws_info['H' + str(j)].value) + '|'
                            + 'cell:' + str(ws_info['I' + str(j)].value) + '|'
                            + 'enviroroment:' + str(ws_info['J' + str(j)].value) + '|'
                            + 'treatment:' + str(ws_info['K' + str(j)].value) + '|'
                            + 'dose:' + str(ws_info['L' + str(j)].value) + '|'
                            + 'time:' + str(ws_info['M' + str(j)].value) + '|'
                            + 'FDR_threshold:' + str(ws_info['N' + str(j)].value) + '|'
                            + 'method:' + str(ws_info['O' + str(j)].value) + '|'
                            + 'comparison:' + str(ws_info['P' + str(j)].value) + '|'
                            + 'strain:' + str(ws_info['Q' + str(j)].value) + '|'
                            + 'statistics:' + str(ws_info['R' + str(j)].value))


    wb_alias.save(info_file_path)


In [4]:
# LOAD START FILES

## PUBLICATION SOURCE
wb = openpyxl.load_workbook(r'../ifpan-GR-database-papers.xlsx')
ws = wb['NPSCs-KO-Cav1']

publication_geneName = [ws.cell(row=i,column=3).value for i in range(2,ws.max_row+1)]
direction = [ws.cell(row=i,column=6).value for i in range(2,ws.max_row+1)]

## MGI
wb_1 = openpyxl.load_workbook(r'../MGI_EntrezGene.xlsx')
ws_1 = wb_1.active

ls_row_10 = [ws_1.cell(row=i,column=10).value for i in range(2,ws_1.max_row+1)]
ls_row_2 = [ws_1.cell(row=i,column=2).value for i in range(2,ws_1.max_row+1)]
ls_row_1 = [ws_1.cell(row=i,column=1).value for i in range(2,ws_1.max_row+1)]  

In [5]:
# LISTS

dictionary = {}
ls_geneDictionaries = []
ls_notResponse = []
ls_notResponse_after = []


In [6]:
# DICTIONARY

for i in range(len(publication_geneName)):
    gene_name = publication_geneName[i]
    direction_string = direction[i]   # => info
    
    if gene_name != '':

        # VARIABLES
        index = i + 1
        gene_list_number = 27
        gene_list_id =  'all_sig_genes_24777604'      # => cluster
        source = 'PMID: 24777604'
        organism = 'mouse'
        print(direction_string)
        alias = ''


        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []

        ls_biomartParameters = biomartParameters(gene_name)
        if not ls_biomartParameters:
            ls_notResponse.append(gene_name)
            print(ls_notResponse)


        for ls in ls_biomartParameters:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
        
        
        # ORTHOLOGS
        ls_biomartHumanOrthologs = biomartHumanOrthologs(gene_name)
        if not ls_biomartHumanOrthologs:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologs[0][0]
        
        temp_gene_dictionary = gene_dictionary(index,
                                               gene_name,
                                               gene_list_number,
                                               gene_list_id,
                                               source,
                                               ensembl_gene_id,
                                               ensembl_transcript_id, 
                                               refseq_mrna,
                                               hgnc_symbol,
                                               organism,
                                               alias,
                                               direction_string)

        ls_geneDictionaries.append(temp_gene_dictionary)
        print(temp_gene_dictionary)





up
{'Index': 1, 'Gene_name': 'Cftr', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000041301', 'Ensembl_transcript_id': 'ENSMUST00000115405|ENSMUST00000129452|ENSMUST00000140532|ENSMUST00000115406|ENSMUST00000140407|ENSMUST00000045706', 'RefSeq_mRNA_id': 'NM_021050', 'Organism': 'mouse', 'HGNC_symbol': 'CFTR', 'Alias': '', 'Info': 'up'}
up
{'Index': 2, 'Gene_name': 'Fkbp5', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000024222', 'Ensembl_transcript_id': 'ENSMUST00000233291|ENSMUST00000177939|ENSMUST00000147716|ENSMUST00000233372|ENSMUST00000153744|ENSMUST00000233917|ENSMUST00000233747|ENSMUST00000114792|ENSMUST00000233307|ENSMUST00000143685|ENSMUST00000233870|ENSMUST00000232690|ENSMUST00000232939|ENSMUST00000079413|ENSMUST00000233102', 'RefSeq_mRNA_id': 'NM_010220', 'Organism': 'mouse', 'HGNC_symbol': 'FKBP5', 'Alias': '', 'In

{'Index': 21, 'Gene_name': 'Pik3r1', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000041417', 'Ensembl_transcript_id': 'ENSMUST00000187009|ENSMUST00000190171|ENSMUST00000185795|ENSMUST00000055518|ENSMUST00000035532|ENSMUST00000189426|ENSMUST00000185701', 'RefSeq_mRNA_id': 'NM_001077495|NM_001024955', 'Organism': 'mouse', 'HGNC_symbol': 'PIK3R1', 'Alias': '', 'Info': 'up'}
up
{'Index': 22, 'Gene_name': 'Gjb6', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000040055', 'Ensembl_transcript_id': 'ENSMUST00000039380|ENSMUST00000162931|ENSMUST00000160703|ENSMUST00000224544', 'RefSeq_mRNA_id': 'NM_001010937|NM_001271663', 'Organism': 'mouse', 'HGNC_symbol': 'GJB6', 'Alias': '', 'Info': 'up'}
up
{'Index': 23, 'Gene_name': 'Tmem229b', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensemb

{'Index': 42, 'Gene_name': 'Zfp189', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000039634', 'Ensembl_transcript_id': 'ENSMUST00000042964|ENSMUST00000107696', 'RefSeq_mRNA_id': 'NM_001289901|NM_145547', 'Organism': 'mouse', 'HGNC_symbol': 'ZNF189', 'Alias': '', 'Info': 'up'}
up
{'Index': 43, 'Gene_name': 'Pdk4', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000019577', 'Ensembl_transcript_id': 'ENSMUST00000019721|ENSMUST00000203347|ENSMUST00000134068', 'RefSeq_mRNA_id': 'NM_013743', 'Organism': 'mouse', 'HGNC_symbol': 'PDK4', 'Alias': '', 'Info': 'up'}
up
{'Index': 44, 'Gene_name': 'Plekhf1', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000074170', 'Ensembl_transcript_id': 'ENSMUST00000098513', 'RefSeq_mRNA_id': 'NM_024413', 'Organism': 'mouse', '

{'Index': 63, 'Gene_name': 'Thrsp', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000035686', 'Ensembl_transcript_id': 'ENSMUST00000043077|ENSMUST00000206613', 'RefSeq_mRNA_id': 'NM_009381', 'Organism': 'mouse', 'HGNC_symbol': 'THRSP', 'Alias': '', 'Info': 'up'}
down
{'Index': 64, 'Gene_name': 'Rpia', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000053604', 'Ensembl_transcript_id': 'ENSMUST00000129244|ENSMUST00000124633|ENSMUST00000066134', 'RefSeq_mRNA_id': 'NM_009075', 'Organism': 'mouse', 'HGNC_symbol': 'RPIA', 'Alias': '', 'Info': 'down'}
down
{'Index': 65, 'Gene_name': 'Nova1', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000021047', 'Ensembl_transcript_id': 'ENSMUST00000219886|ENSMUST00000218829|ENSMUST00000021438|ENSMUST00000220258|ENSMUST00

{'Index': 84, 'Gene_name': 'Gcnt2', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000021360', 'Ensembl_transcript_id': 'ENSMUST00000225759|ENSMUST00000225996|ENSMUST00000110191|ENSMUST00000069958|ENSMUST00000067778|ENSMUST00000223570', 'RefSeq_mRNA_id': 'NM_133219|NM_023887|NM_008105', 'Organism': 'mouse', 'HGNC_symbol': 'GCNT2', 'Alias': '', 'Info': 'up'}
up
{'Index': 85, 'Gene_name': 'Cacna1h', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000024112', 'Ensembl_transcript_id': 'ENSMUST00000162820|ENSMUST00000159623|ENSMUST00000161658|ENSMUST00000161035|ENSMUST00000159610|ENSMUST00000159052|ENSMUST00000159331|ENSMUST00000161454|ENSMUST00000078496|ENSMUST00000159048', 'RefSeq_mRNA_id': 'NM_001163691|NM_021415', 'Organism': 'mouse', 'HGNC_symbol': 'CACNA1H', 'Alias': '', 'Info': 'up'}
up
{'Index': 86, 'Gene_name': 'Rgs2', 'Gene_li

...

{'Index': 860, 'Gene_name': 'Csk', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000032312', 'Ensembl_transcript_id': 'ENSMUST00000216934|ENSMUST00000213660|ENSMUST00000215396|ENSMUST00000215958|ENSMUST00000034863|ENSMUST00000217128|ENSMUST00000217314|ENSMUST00000216979|ENSMUST00000213943', 'RefSeq_mRNA_id': 'NM_007783|NM_001304761', 'Organism': 'mouse', 'HGNC_symbol': 'CSK', 'Alias': '', 'Info': 'down'}
up
{'Index': 861, 'Gene_name': 'Asap1', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000022377', 'Ensembl_transcript_id': 'ENSMUST00000175710|ENSMUST00000110114|ENSMUST00000176516|ENSMUST00000176427|ENSMUST00000175963|ENSMUST00000176909|ENSMUST00000177475|ENSMUST00000175799|ENSMUST00000175793|ENSMUST00000177035|ENSMUST00000110115|ENSMUST00000176014|ENSMUST00000177374|ENSMUST00000176358|ENSMUST00000023008|ENSMUST00000177083|ENSM

{'Index': 871, 'Gene_name': 'Adprhl2', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'mouse', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
down
['Galntl1', '1110067D22Rik', 'Cxcr7', 'Fbxo18', 'Fam101b', '1110007C09Rik', 'Gm9971', 'Ndnl2', 'Fam176a', 'Mpp5', 'Mycl1', 'Epb4.1l4b', 'D16H22S680E', 'Gm16516', 'Fam101b', 'Gm9971', 'Ndnl2', 'Fam176a', '1110067D22Rik', '1810011O10Rik', 'Pnp /// Pnp2', 'A030009H04Rik', 'Ctgf', 'Fbxo18', '1110067D22Rik', 'Fam101b', 'Gm9971', 'Ndnl2', 'Fam176a', 'Fbxo18', '1110007C09Rik', 'Galntl1', 'Pnp_Pnp2', 'D16H22S680E', 'Mycl1', 'Mpp5', 'Ctgf', 'A030009H04Rik', '1810011O10Rik', 'Cxcr7', 'Clca1_Clca2', 'Mtap7', 'Zfp191', 'St5', '4931408A02Rik', 'Fam134b', 'Dynlt1a_Dynlt1b_Dynlt1c_Dynlt1e_Dynlt1f', 'Jub', 'LOC100505067_Zfp82', 'D11Wsu99e', 'Epb4.1l4b', 'Ncrna00081', 'Higd1c_Mettl7a2', '4933426M11Rik', 'Vipar', datetime.time

{'Index': 887, 'Gene_name': 'Phyhipl', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000037747', 'Ensembl_transcript_id': 'ENSMUST00000160127|ENSMUST00000159025|ENSMUST00000163054|ENSMUST00000046513|ENSMUST00000162251|ENSMUST00000161687|ENSMUST00000162571|ENSMUST00000162793|ENSMUST00000162144|ENSMUST00000162470', 'RefSeq_mRNA_id': 'NM_001162846|NM_178621', 'Organism': 'mouse', 'HGNC_symbol': 'PHYHIPL', 'Alias': '', 'Info': 'up'}
down
{'Index': 888, 'Gene_name': 'Ggt7', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000027603', 'Ensembl_transcript_id': 'ENSMUST00000138033|ENSMUST00000147601|ENSMUST00000029131|ENSMUST00000148296|ENSMUST00000176117', 'RefSeq_mRNA_id': 'NM_001362900|NM_144786|NM_001362898', 'Organism': 'mouse', 'HGNC_symbol': 'GGT7', 'Alias': '', 'Info': 'down'}
up
{'Index': 889, 'Gene_name': 'Etaa1', 'Gene_list_numb

{'Index': 899, 'Gene_name': 'Tsc22d4', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'mouse', 'HGNC_symbol': '', 'Alias': '', 'Info': 'down'}
down
{'Index': 900, 'Gene_name': 'Fzd9', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000049551', 'Ensembl_transcript_id': 'ENSMUST00000062572', 'RefSeq_mRNA_id': 'NM_010246', 'Organism': 'mouse', 'HGNC_symbol': 'FZD9', 'Alias': '', 'Info': 'down'}
up
{'Index': 901, 'Gene_name': 'Gnpda1', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000052102', 'Ensembl_transcript_id': 'ENSMUST00000237313|ENSMUST00000063814|ENSMUST00000139885|ENSMUST00000237487|ENSMUST00000235590|ENSMUST00000153291|ENSMUST00000129062', 'RefSeq_mRNA_id': 'NM_011937', 'Organism': 'mouse',

{'Index': 917, 'Gene_name': 'Tesk1', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000028458', 'Ensembl_transcript_id': 'ENSMUST00000060864|ENSMUST00000126350|ENSMUST00000156810|ENSMUST00000138981|ENSMUST00000137178', 'RefSeq_mRNA_id': 'NM_011571', 'Organism': 'mouse', 'HGNC_symbol': 'TESK1', 'Alias': '', 'Info': 'down'}
down
{'Index': 918, 'Gene_name': 'Pygo2', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source': 'PMID: 24777604', 'Ensembl_gene_id': 'ENSMUSG00000047824', 'Ensembl_transcript_id': 'ENSMUST00000133680|ENSMUST00000060061|ENSMUST00000125830|ENSMUST00000146037|ENSMUST00000136573|ENSMUST00000107413', 'RefSeq_mRNA_id': 'NM_001293766|NM_001293763|NM_001293767|NM_001293768|NM_026869', 'Organism': 'mouse', 'HGNC_symbol': 'PYGO2', 'Alias': '', 'Info': 'down'}
down
{'Index': 919, 'Gene_name': 'Magi1', 'Gene_list_number': 27, 'Gene_list_index': 'all_sig_genes_24777604', 'Source

In [9]:
print(ls_notResponse)

['Galntl1', '1110067D22Rik', 'Cxcr7', 'Fbxo18', 'Fam101b', '1110007C09Rik', 'Gm9971', 'Ndnl2', 'Fam176a', 'Mpp5', 'Mycl1', 'Epb4.1l4b', 'D16H22S680E', 'Gm16516', 'Fam101b', 'Gm9971', 'Ndnl2', 'Fam176a', '1110067D22Rik', '1810011O10Rik', 'Pnp /// Pnp2', 'A030009H04Rik', 'Ctgf', 'Fbxo18', '1110067D22Rik', 'Fam101b', 'Gm9971', 'Ndnl2', 'Fam176a', 'Fbxo18', '1110007C09Rik', 'Galntl1', 'Pnp_Pnp2', 'D16H22S680E', 'Mycl1', 'Mpp5', 'Ctgf', 'A030009H04Rik', '1810011O10Rik', 'Cxcr7', 'Clca1_Clca2', 'Mtap7', 'Zfp191', 'St5', '4931408A02Rik', 'Fam134b', 'Dynlt1a_Dynlt1b_Dynlt1c_Dynlt1e_Dynlt1f', 'Jub', 'LOC100505067_Zfp82', 'D11Wsu99e', 'Epb4.1l4b', 'Ncrna00081', 'Higd1c_Mettl7a2', '4933426M11Rik', 'Vipar', datetime.time(0, 0), 'Fam100b', 'Selm', '5430435G22Rik', 'Zfp161', 'Loh12cr1', 'Fam188a', 'Adnp_Dpm1', 'Muted', 'Ftsjd1', 'Setd8', '1110054M08Rik', 'Diap1', 'Fam58b', 'Fam126a', 'Gatsl2', 'Fam48a', 'Myst3', 'Cxx1a_Cxx1b', 'Mll1', '2310044G17Rik', 'D530037H12Rik', 'Pvrl2', 'Klraq1', 'Adprhl2', '

In [10]:
# SCORES 

### from DICTIONARY to file
data = ls_geneDictionaries
df = pd.DataFrame.from_dict(data)
df.to_excel(r'.\Dictionary.xlsx', index=False)

### from ls_notResponse to file
data1 = ls_notResponse
df1 = pd.DataFrame.from_dict(data1)
df1.to_excel(r'.\notResponse.xlsx', index=False)

# LOAD 
# notRESPONSE
wb_2 = openpyxl.load_workbook(r'.\notResponse.xlsx')
ws_2 = wb_2.active
ls_notResponse_v2 = [ws_2.cell(row=i,column=1).value for i in range(2,ws_2.max_row+1)]


### from alias_and_official to file
data2 = alias_and_official(ls_notResponse, ls_row_10, ls_row_2)
df2 = pd.DataFrame.from_dict(data2)
df2.to_excel(r'.\responsewithMGI.xlsx', index=False)


# LOAD responsewithMGI
wb_3 = openpyxl.load_workbook(r'.\responsewithMGI.xlsx')
ws_3 = wb_3.active
mgi_id = [ws_3.cell(row=i,column=3).value for i in range(3,ws_3.max_row+1)]
gene_name_v2 = [ws_3.cell(row=i,column=1).value for i in range(3,ws_3.max_row+1)]

['1110067d22rik', 50657, 'MGI:1916114']
['1110007c09rik', 50085, 'MGI:1915730']
['1110067d22rik', 50657, 'MGI:1916114']
['1810011o10rik', 50962, 'MGI:1916318']
['1110067d22rik', 50657, 'MGI:1916114']
['1110007c09rik', 50085, 'MGI:1915730']
['1810011o10rik', 50962, 'MGI:1916318']
['jub', 37229, 'MGI:1341886']
['d11wsu99e', 24669, 'MGI:106351']
['4933426m11rik', 77986, 'MGI:2444661']
['1110054m08rik', 50623, 'MGI:1916091']
['2310044g17rik', 55231, 'MGI:1919185']
['d2wsu81e', 24910, 'MGI:106544']
['mesdc1', 65626, 'MGI:1891420']


In [11]:
print (ls_notResponse)

['Galntl1', '1110067D22Rik', 'Cxcr7', 'Fbxo18', 'Fam101b', '1110007C09Rik', 'Gm9971', 'Ndnl2', 'Fam176a', 'Mpp5', 'Mycl1', 'Epb4.1l4b', 'D16H22S680E', 'Gm16516', 'Fam101b', 'Gm9971', 'Ndnl2', 'Fam176a', '1110067D22Rik', '1810011O10Rik', 'Pnp /// Pnp2', 'A030009H04Rik', 'Ctgf', 'Fbxo18', '1110067D22Rik', 'Fam101b', 'Gm9971', 'Ndnl2', 'Fam176a', 'Fbxo18', '1110007C09Rik', 'Galntl1', 'Pnp_Pnp2', 'D16H22S680E', 'Mycl1', 'Mpp5', 'Ctgf', 'A030009H04Rik', '1810011O10Rik', 'Cxcr7', 'Clca1_Clca2', 'Mtap7', 'Zfp191', 'St5', '4931408A02Rik', 'Fam134b', 'Dynlt1a_Dynlt1b_Dynlt1c_Dynlt1e_Dynlt1f', 'Jub', 'LOC100505067_Zfp82', 'D11Wsu99e', 'Epb4.1l4b', 'Ncrna00081', 'Higd1c_Mettl7a2', '4933426M11Rik', 'Vipar', datetime.time(0, 0), 'Fam100b', 'Selm', '5430435G22Rik', 'Zfp161', 'Loh12cr1', 'Fam188a', 'Adnp_Dpm1', 'Muted', 'Ftsjd1', 'Setd8', '1110054M08Rik', 'Diap1', 'Fam58b', 'Fam126a', 'Gatsl2', 'Fam48a', 'Myst3', 'Cxx1a_Cxx1b', 'Mll1', '2310044G17Rik', 'D530037H12Rik', 'Pvrl2', 'Klraq1', 'Adprhl2', '

In [12]:
# SECOND DICTIONARY

for i in range(len(mgi_id)):
    mgi_ID = mgi_id[i]
    gene_name = gene_name_v2[i]
    print(mgi_ID)
    
    if gene_name != '':

        # variables
        alias = ''
        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []
        

        ls_biomartParameters_mgi = biomartParameters_mgi(mgi_ID)
        
        print(ls_biomartParameters_mgi)
        
        if not ls_biomartParameters_mgi:
            ls_notResponse_after.append(mgi_ID)


        for ls in ls_biomartParameters_mgi:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
         
        
        ls_biomartHumanOrthologs_mgi = biomartHumanOrthologs_mgi(mgi_ID)
        if not ls_biomartHumanOrthologs_mgi:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologs_mgi[0][0]       
        
        
        for k in range(len(ls_geneDictionaries)):
            temp = str(ls_geneDictionaries[k]['Gene_name']).lower()

            if temp == gene_name:
                print('response')
                ls_geneDictionaries[k]['Ensembl_gene_id'] = ensembl_gene_id
                ls_geneDictionaries[k]['Ensembl_transcript_id'] = ensembl_transcript_id
                ls_geneDictionaries[k]['RefSeq_mRNA_id'] = refseq_mrna
                ls_geneDictionaries[k]['HGNC_symbol'] = hgnc_symbol
                


SOCS1
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
TSC22D3
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
SPRY1
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000

TIPARP
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
BCL6
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
SNAI2
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG0000003

TNFAIP3
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
STRA8
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
PARD6B
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG0000

FGD4
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
GENX-3414
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
FOSL2
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG0000

FAM105A
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
ZFAND5
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
PNRC1
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG0000

...

None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG0000003929

None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG0000003929

None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG0000003929

None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG0000003929

None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG00000039298', 'ENSMUST00000126416', ''], ['ENSMUSG00000039298', 'ENSMUST00000138561', ''], ['ENSMUSG00000039298', 'ENSMUST00000140108', '']]
None
[['ENSMUSG00000039298', 'ENSMUST00000124251', ''], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_001313762'], ['ENSMUSG00000039298', 'ENSMUST00000144099', 'NM_145990'], ['ENSMUSG00000039298', 'ENSMUST00000076541', ''], ['ENSMUSG0000003929

In [14]:
# SCORES 

### from SECOND DICTIONARY to file

data3 = ls_geneDictionaries
df3 = pd.DataFrame.from_dict(data3)
df3.to_excel(r'.\secondDictionary.xlsx', index=False)
df3.to_csv(r'.\secondDictionary.tsv', sep="\t")

data4 = ls_notResponse_after
df4 = pd.DataFrame.from_dict(data4)
df4.to_excel(r'.\ls_notResponse_after.xlsx', index=False)



In [5]:
# Add ALIAS

mgi_file_path = r'..\MGI_EntrezGene.xlsx'
dictionary_file_path = r'.\secondDictionary.xlsx'
save_file_path = r'.\withAlias.xlsx'


updateCellswithAlias(mgi_file_path, dictionary_file_path, save_file_path)


In [8]:
# Add INFO

source_file_path = r'..\ifpan-GR-database-papers.xlsx'
source_sheet = 'NPSCs-KO-Cav1'
alias_file_path = '.\withAlias.xlsx'
info_file_path = r'.\withINFO.xlsx'

updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path)


cftr cftr
fkbp5 fkbp5
hif3a hif3a
tsc22d3 tsc22d3
klf9 klf9
mertk mertk
tprn tprn
zbtb16 zbtb16
per1 per1
errfi1 errfi1
smox smox
galntl1 galntl1
slc10a6 slc10a6
sesn1 sesn1
nedd9 nedd9
pip4k2a pip4k2a
bcl2l1 bcl2l1
per2 per2
gbx2 gbx2
nek7 nek7
pik3r1 pik3r1
gjb6 gjb6
tmem229b tmem229b
ptk2b ptk2b
lrrc8a lrrc8a
cebpd cebpd
ada ada
cables1 cables1
ptprj ptprj
chst2 chst2
rhob rhob
camkk1 camkk1
pim3 pim3
lpin1 lpin1
kcnn2 kcnn2
phf13 phf13
1110067d22rik 1110067d22rik
rcan2 rcan2
zfp281 zfp281
arid5b arid5b
cxcr7 cxcr7
zfp189 zfp189
pdk4 pdk4
plekhf1 plekhf1
bcl6 bcl6
taf8 taf8
fbxo18 fbxo18
klf13 klf13
greb1 greb1
phyhd1 phyhd1
cebpb cebpb
elmo1 elmo1
tob2 tob2
fam101b fam101b
sema4b sema4b
peg3 peg3
smc5 smc5
rhpn2 rhpn2
f3 f3
usp2 usp2
slc25a33 slc25a33
ezr ezr
thrsp thrsp
rpia rpia
nova1 nova1
1110007c09rik 1110007c09rik
jdp2 jdp2
gm9971 gm9971
dlg5 dlg5
arl4d arl4d
hspb1 hspb1
ahcyl1 ahcyl1
gab1 gab1
ndnl2 ndnl2
zfp566 zfp566
cspg4 cspg4
rhou rhou
sh3rf1 sh3rf1
slc39a14 slc39a14
no

In [ ]:
                                ''' + '|'
                                 + '|'
                                +'TTEST.NEURO.ALD:' + str(ws_mgi[S_cel].value) + '|'
                                +'TTEST.NEURO.DHT:' + str(ws_mgi[T_cel].value) + '|'
                                +'TTEST.NEURO.PRG:' + str(ws_mgi[U_cel].value) + '|'
                                +'TTEST.NEURO.ESR:' + str(ws_mgi[V_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DEX:' + str(ws_mgi[W_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ALD:' + str(ws_mgi[X_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DHT:' + str(ws_mgi[Y_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.PRG:' + str(ws_mgi[Z_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ESR:' + str(ws_mgi[AA_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DEX:' + str(ws_mgi[AB_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ALD:' + str(ws_mgi[AC_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DHT:' + str(ws_mgi[AD_cel].value) + '|'
                                +'FDR_TTEST.NEURO.PRG:' + str(ws_mgi[AE_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ESR:' + str(ws_mgi[AF_cel].value) + '|'
                                +'FOLD.ASTRO.DEX:' + str(ws_mgi[AG_cel].value) + '|'
                                +'FOLD.ASTRO.ALD:' + str(ws_mgi[AH_cel].value) + '|'
                                +'FOLD.ASTRO.DHT:' + str(ws_mgi[AI_cel].value) + '|'
                                +'FOLD.ASTRO.PRG:' + str(ws_mgi[AJ_cel].value) + '|'
                                +'FOLD.ASTRO.ESR:' + str(ws_mgi[AK_cel].value) + '|'
                                +'FOLD.NEURO.DEX:' + str(ws_mgi[AL_cel].value) + '|'
                                +'FOLD.NEURO.ALD:' + str(ws_mgi[AM_cel].value) + '|'
                                +'FOLD.NEURO.DHT:' + str(ws_mgi[AN_cel].value) + '|'
                                +'FOLD.NEURO.PRG:' + str(ws_mgi[AO_cel].value) + '|'
                                +'FOLD.NEURO.ESR:' + str(ws_mgi[AP_cel].value))'''